In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
from mypdata import get_train_xy,resample_xy,get_split,get_c_s_data

In [2]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
test_s,test_c = get_c_s_data(df_test)

Xo,yo,df2 = get_train_xy(df)
s_df,c_df=get_c_s_data(df2)
# print(s_df,c_df)
Xc,yc = c_df.drop(["target"],axis=1),c_df["target"]
Xs,ys = s_df.drop(["target"],axis=1),s_df["target"]

list2= ['36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95']
list2= ['36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95']


In [5]:
print(test_c,Xc)

               0         1         2         3         4         5         6  \
0      -0.715024 -0.201341 -0.678386 -0.450571 -0.120901 -0.356814 -0.353468   
1       1.689397  0.102079  0.556177  0.603863 -1.075069 -0.811886  1.991656   
2       0.996288 -0.338883 -0.707930 -1.677825  0.159960 -0.533892 -1.748014   
3       0.292867  1.340973  0.073957  0.016702 -2.531774 -0.049783 -0.742053   
4      -0.626375  1.537576 -0.239537 -1.687296  0.673543 -0.869008 -0.548668   
...          ...       ...       ...       ...       ...       ...       ...   
194325  1.708959  0.244682  0.416518 -0.325758 -1.876907  0.484784  0.078695   
194326  0.002344 -0.747105 -1.149667  3.324977  0.948170  1.737184  0.149512   
194327  0.402384  0.107534 -0.403283 -0.201043 -0.892297  1.075521  2.255857   
194328 -0.610692 -1.104640 -0.009219  0.941540  0.086929  0.279145  0.582584   
194329 -0.043290  1.678720  0.275665  0.572649 -1.048620  1.060764 -1.077978   

               7  8         9  ...     

In [6]:
scale_pos_weight_s = (len(ys) - sum(ys)) / sum(ys)
scale_pos_weight_c = (len(yc) - sum(yc)) / sum(yc)
print("scale_pos_weight =", scale_pos_weight_s,scale_pos_weight_c)

scale_pos_weight = 4.329703062459275 4.329703062459275


In [7]:
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2, stratify=yc, random_state=42)
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, test_size=0.2, stratify=ys, random_state=42)

In [8]:
print(len(Xc_train),len(yc_train))

91611 91611


In [9]:
# XGBoost 處理連續型
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    gamma=0.1,
    tree_method='hist',
    random_state=42,
    scale_pos_weight=scale_pos_weight_c,
    use_label_encoder=False,
    eval_metric='logloss'
)

# Random Forest 處理結構化
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

xgb_model.fit(Xc, yc)
rf_model.fit(Xs, ys)

/opt/miniconda3/envs/nthu_ml/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [00:17:53] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=200,
                       n_jobs=-1, random_state=42)

In [10]:
# 將模型包成 pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Meta 模型使用邏輯回歸
meta_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)

# # 用 stacking 將兩個模型整合
# stacked_model = StackingClassifier(
#     estimators=[
#         ('xgb', xgb_model),
#         ('rf', rf_model)
#     ],
#     final_estimator=meta_model,
#     passthrough=True,
#     n_jobs=-1
# )

# 用 base 模型對 test 資料做預測（這是給 meta model 的輸入）
cont_pred_train = xgb_model.predict_proba(Xc)[:, 1]
struct_pred_train = rf_model.predict_proba(Xs)[:, 1]

# meta model 訓練
meta_X_train = pd.DataFrame({
    'cont_pred': cont_pred_train,
    'struct_pred': struct_pred_train
})

In [11]:
print((yc == ys).all())

True


In [12]:
meta_model = LogisticRegression()
meta_model.fit(meta_X_train, yc)

LogisticRegression()

In [13]:
print(test_s)

         36   37   38   39   40   41   42   43   44   45  ...   86   87   88  \
0       0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1       1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4       0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...     ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
194325  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
194326  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  1.0  0.0  0.0   
194327  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  0.0   
194328  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0  0.0  1.0   
194329  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

         89   90   91   92   93   94   

In [14]:
cont_pred_test = xgb_model.predict_proba(test_c)[:, 1]
struct_pred_test = rf_model.predict_proba(test_s)[:, 1]

meta_X_test = pd.DataFrame({
    'cont_pred': cont_pred_test,
    'struct_pred': struct_pred_test
})

In [15]:
final_pred = meta_model.predict(meta_X_test)

In [16]:
results1 = pd.DataFrame({
    "index": df_test["index"],  # 保留原始 ID
    "target": final_pred
})

# results2 = pd.DataFrame({
#     "index": df_test["index"],  # 保留原始 ID
#     "target": y2_pred
# })

results1.to_csv("predictionsxx.csv", index=False)

In [13]:
# 合併連續與結構化資料
from numpy import concatenate

X_train_full = pd.concat([Xc_train.reset_index(drop=True), Xs_train.reset_index(drop=True)], axis=1)
X_test_full = pd.concat([Xc_test.reset_index(drop=True), Xs_test.reset_index(drop=True)], axis=1)

# 訓練模型
stacked_model.fit(X_train_full, y_train)

# 預測
y_pred = stacked_model.predict(X_test_full)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


XGBoostError: [23:34:20] /Users/runner/work/xgboost/xgboost/src/data/data.cc:542: Check failed: this->labels.Size() % this->num_row_ == 0 (91611 vs. 0) : Incorrect size for labels: (91611,1) v.s. 124860
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000012d68dbfc dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000012d7f1944 xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const*, xgboost::StringView, xgboost::Json) + 2580
  [bt] (2) 3   libxgboost.dylib                    0x000000012d7f0da4 xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 464
  [bt] (3) 4   libxgboost.dylib                    0x000000012d6a54a8 XGDMatrixSetInfoFromInterface + 228
  [bt] (4) 5   libffi.8.dylib                      0x000000010644804c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib                      0x0000000106445834 ffi_call_int + 1404
  [bt] (6) 7   _ctypes.cpython-310-dar